<a href="https://colab.research.google.com/github/morgan-h-4/511FP/blob/main/Prune_and_Quantize.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import tensorflow as tf

print(f"NumPy version: {np.__version__}")
print(f"TensorFlow version: {tf.__version__}")

NumPy version: 2.0.2
TensorFlow version: 2.18.0


In [4]:
import tempfile
import os
import time

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import keras
# import tensorflow_model_optimization as tfmot

In [5]:
from tensorflow.keras import datasets, layers, models

In [3]:
!pip install ultralytics
# !pip install roboflow opencv-python numpy torch torchvision

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.8/974.8 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 90.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 29.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 53.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstall

In [6]:
import torch
import torch.nn.utils.prune as prune #main functionality
from ultralytics import YOLO

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [7]:
def prune_model(model, amount):
    for module in model.modules():
        if isinstance(module, torch.nn.Conv2d):
            prune.l1_unstructured(module, name="weight", amount=amount) #on individual weights
            prune.remove(module, "weight")
    return model

path = '/content/drive/MyDrive/School/Year 4/DIGITAL 500/ENDG_511/Project/morgan_attempt'
model = YOLO(f"{path}/construction_ppe_yolov8_trained_model_25epoch.pt")
results = model.val(data=f"{path}/costruction_ppe.v3i.yolov8/data.yaml")
print(f"mAP50-95: {results.box.map}") # overall mean error precision

torch_model = model.model
print(torch_model)

print("Pruning model...")
pruned_torch_model = prune_model(torch_model, 0.1) #pruning sparsity/ amount set here
print("Model pruned.")


Ultralytics 8.3.108 🚀 Python-3.11.12 torch-2.6.0+cu124 CPU (Intel Xeon 2.20GHz)
Model summary (fused): 72 layers, 3,007,403 parameters, 0 gradients, 8.1 GFLOPs


100%|██████████| 755k/755k [00:00<00:00, 3.38MB/s]
val: Scanning /content/drive/MyDrive/School/Year 4/DIGITAL 500/ENDG_511/Project/morgan_attempt/costruction_ppe.v3i.yolov8/valid/labels.cache... 302 images, 0 backgrounds, 0 corrupt: 100%|██████████| 302/302 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 19/19 [01:38<00:00,  5.20s/it]


                   all        302       1895      0.783      0.818      0.812      0.549
                 boots         64        150      0.847      0.833      0.876      0.583
                  face        262        321      0.666      0.928      0.869      0.439
                gloves        134        263      0.473      0.445      0.456      0.187
               hardhat        185        221      0.943       0.95      0.978      0.731
               noboots         11         20      0.823        0.7      0.724      0.418
              nogloves         77        126      0.518      0.627      0.502      0.204
                person        272        332      0.874      0.961      0.955      0.788
               rubboot         74        146      0.969      0.966      0.975      0.798
                  vest        238        316      0.938      0.956      0.968      0.791
Speed: 5.2ms preprocess, 280.6ms inference, 0.0ms loss, 1.0ms postprocess per image
Results saved to runs/dete

In [8]:
model.model = pruned_torch_model

print("Saving pruned model...")

model.save(f"{path}/yolov8s_pruned.pt")

print("Pruned Model Saved")

Saving pruned model...
Pruned Model Saved


In [9]:
model = YOLO(f"{path}/yolov8s_pruned.pt")
results = model.val(data=f"{path}/costruction_ppe.v3i.yolov8/data.yaml")
print(f"mAP50-95: {results.box.map}")

Ultralytics 8.3.108 🚀 Python-3.11.12 torch-2.6.0+cu124 CPU (Intel Xeon 2.20GHz)


val: Scanning /content/drive/MyDrive/School/Year 4/DIGITAL 500/ENDG_511/Project/morgan_attempt/costruction_ppe.v3i.yolov8/valid/labels.cache... 302 images, 0 backgrounds, 0 corrupt: 100%|██████████| 302/302 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 19/19 [01:26<00:00,  4.53s/it]


                   all        302       1895      0.597      0.757      0.717      0.433
                 boots         64        150      0.635      0.793      0.799      0.503
                  face        262        321      0.574      0.872      0.708      0.315
                gloves        134        263      0.569      0.221      0.414      0.163
               hardhat        185        221      0.394       0.91      0.699      0.378
               noboots         11         20      0.693       0.65      0.645      0.353
              nogloves         77        126      0.501      0.556      0.467      0.175
                person        272        332      0.703      0.919      0.903      0.707
               rubboot         74        146      0.474      0.959      0.884      0.592
                  vest        238        316      0.829      0.934      0.932      0.716
Speed: 5.4ms preprocess, 263.0ms inference, 0.0ms loss, 0.8ms postprocess per image
Results saved to runs/dete

Now let's try and Quantize

In [10]:
!pip install torch torchvision
import torch
import torchvision.models as models


In [11]:
# model = torch.load(f"{path}/yolov8s_pruned.pt")
# model.eval()
model = YOLO(f"{path}/yolov8s_pruned.pt")

# Access the underlying torch.nn.Module
torch_model = model.model

In [12]:
quantized_model = torch.quantization.quantize_dynamic(
    torch_model, # Pass the torch.nn.Module object,
    {torch.nn.Linear},  # layers to quantize
    dtype=torch.qint8)


In [13]:
torch.save(quantized_model, f"{path}/yolov8s_pruned_quantized.pt")
# files.download(f"{path}/yolov8s_pruned_quantized.pt")

In [14]:
import os

original_size = os.path.getsize(f"{path}/construction_ppe_yolov8_trained_model_25epoch.pt") / 1e6  # in MB
pruned_size = os.path.getsize(f"{path}/yolov8s_pruned.pt") / 1e6  # in MB
quantized_size = os.path.getsize(f"{path}/yolov8s_pruned_quantized.pt") / 1e6  # in MB

print(f"Original model size: {original_size:.2f} MB")
print(f"Pruned model size: {pruned_size:.2f} MB")
print(f"Quantized model size: {quantized_size:.2f} MB")
# print(f"Size reduction: {100 * (original_size - quantized_size) / original_size:.2f}%")


Original model size: 6.28 MB
Pruned model size: 6.16 MB
Quantized model size: 12.23 MB


In [15]:
# Lets try Static Quantization now
model.eval()
model.qconfig = torch.quantization.get_default_qconfig('fbgemm')
torch.quantization.prepare(model, inplace=True)

/usr/local/lib/python3.11/dist-packages/torch/ao/quantization/observer.py:229: UserWarning: Please use quant_min and quant_max to specify the range for observers.                     reduce_range will be deprecated in a future release of PyTorch.
  warnings.warn(


YOLO(
  (model): DetectionModel(
    (model): Sequential(
      (0): Conv(
        (conv): Conv2d(
          3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1)
          (activation_post_process): HistogramObserver(min_val=inf, max_val=-inf)
        )
        (act): SiLU(inplace=True)
      )
      (1): Conv(
        (conv): Conv2d(
          16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1)
          (activation_post_process): HistogramObserver(min_val=inf, max_val=-inf)
        )
        (act): SiLU(inplace=True)
      )
      (2): C2f(
        (cv1): Conv(
          (conv): Conv2d(
            32, 32, kernel_size=(1, 1), stride=(1, 1)
            (activation_post_process): HistogramObserver(min_val=inf, max_val=-inf)
          )
          (act): SiLU(inplace=True)
        )
        (cv2): Conv(
          (conv): Conv2d(
            48, 32, kernel_size=(1, 1), stride=(1, 1)
            (activation_post_process): HistogramObserver(min_val=inf, max_val=-inf)
          )
 

In [16]:
# Example: Run a few batches through the model
for i in range(10):
    sample_input = torch.randn(1, 3, 224, 224)  # adjust to your model
    model(sample_input)


WARNING ⚠️ torch.Tensor inputs should be normalized 0.0-1.0 but max value is 4.420313835144043. Dividing input by 255.
0: 224x224 (no detections), 131.7ms
Speed: 0.0ms preprocess, 131.7ms inference, 3.0ms postprocess per image at shape (1, 3, 224, 224)

WARNING ⚠️ torch.Tensor inputs should be normalized 0.0-1.0 but max value is 4.849902629852295. Dividing input by 255.
0: 224x224 (no detections), 209.7ms
Speed: 0.0ms preprocess, 209.7ms inference, 1.4ms postprocess per image at shape (1, 3, 224, 224)

WARNING ⚠️ torch.Tensor inputs should be normalized 0.0-1.0 but max value is 4.611741065979004. Dividing input by 255.
0: 224x224 (no detections), 162.0ms
Speed: 0.0ms preprocess, 162.0ms inference, 1.3ms postprocess per image at shape (1, 3, 224, 224)

WARNING ⚠️ torch.Tensor inputs should be normalized 0.0-1.0 but max value is 4.729465007781982. Dividing input by 255.
0: 224x224 (no detections), 138.9ms
Speed: 0.0ms preprocess, 138.9ms inference, 1.3ms postprocess per image at shape (

In [18]:
torch.quantization.convert(model, inplace=True)
# torch.save(model, f"{path}/static_quantized_model.pt")
model.save(f"{path}/static_quantized_model.pt")
# files.download("static_quantized_model.pt")

In [19]:
static_quantized_size = os.path.getsize(f"{path}/static_quantized_model.pt") / 1e6  # in MB

print(f"Original model size: {original_size:.2f} MB")
print(f"Pruned model size: {pruned_size:.2f} MB")
print(f"Quantized model size: {quantized_size:.2f} MB")
print(f"Static Quantized model size: {static_quantized_size:.2f} MB")

Original model size: 6.28 MB
Pruned model size: 6.16 MB
Quantized model size: 12.23 MB
Static Quantized model size: 3.23 MB
